In [ ]:
savepath="insert/your/path"
taxname="microbes"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#the distance file with all pairwise tree branch length calculations
df_mp=pd.read_feather("microbes_distances_99OTU.feather", use_threads=True)

df_mp.index=df_mp.columns


#so we made Our first file with Distances now. Let's go on to figure out the distribtuion!


rn2=df_mp.sample(n=3000,random_state=33)

rn1=rn2.values.tolist()

random.seed(123)




a = np.array([])

for i in range(3000):
    b = random.sample(rn1[i],3000)
    a=np.append(a,b)

#create a historgram with the approximate distance distribution and save it

histo=np.histogram(a, bins=50)

plt.hist(a,bins=50)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.hist(a, bins=50)
fig1.savefig(savepath+taxname+"/plots/distplot_"+taxname+"_.png")

#MANUAL STEP: Check distirbution

#IMPORTANT: Here we set the limit of which percentile we want to keep: E.G exclude the far right tail of the distribution, where we sometimes find wrongly 
#annotated taxonomic annotations or OTUs that don't belong th the phylum we are investigating.

per=np.percentile(a,97)


val=per

a1=a[a<val]

#the distrubtion excluding the right tails =3%) of distances, since we may not always be able to fill this bin (and it is uninformative)

histo=np.histogram(a1, bins=50)

fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.hist(a1, bins=50)
fig2.savefig(savepath+taxname+"/plots/distplot_"+taxname+"_corr.png")

weights=histo[0]



weights=(1/histo[0])/(sum(1/histo[0]))

hi=histo[1]

weights=weights*(1/max(weights))



#need to add a check for duplicates and remove them




#OLD VERSION

#convert dataframe into numpy obj

xy=df_mp.to_numpy()



#select our OTUs, filling each bin with 500 OTUs

otus=df_mp.index





globall=[]
otu_count_dict={}
for binnr in range(len(histo[1])-1):


    vals=[]
    arr=np.nonzero((xy>histo[1][binnr])&(xy<histo[1][binnr+1]))

    np.random.state=33

    while len(vals)<500:
        #select random otus
        n=np.random.randint(0,len(arr[0]))
        col_ind=arr[0][n]
        row_ind=arr[1][n]
        #get their identifiers
        otu1=otus[col_ind]
        otu2=otus[row_ind]
        #calculate their tree branch distance
        distance=xy[col_ind,row_ind]
        
        #count the occurences of each OLTU to avoid them being used in too many computations
        if otu1 not in otu_count_dict.keys():
            otu_count_dict[otu1]=1
        else:
            otu_count_dict[otu1]+=1
        if otu2 not in otu_count_dict.keys():
            otu_count_dict[otu2]=1
        else:
            otu_count_dict[otu2]+=1
        #max 10 times an OTU can be used
        if otu_count_dict[otu1] <10 and otu_count_dict[otu2] < 10:
            #make sure we don't have duplicate pairs
            if (otu1+ " " + otu2+ " " + str(distance)) not in vals: 

                    vals.append(otu1+ " " + otu2+ " " + str(distance))
    #attach to global vals variable
    globall.append(vals)
    print(str(binnr)+" done")
#lets set the pandas display value higher to observe the correct branch lengths

vals=[]
for x in globall:
    for y in x:
        vals.append(y)


pd.set_option("display.precision", 10)   

dfdf=pd.DataFrame(columns=["Distance"])

dfdf["Distance"]=vals


dfdf=dfdf["Distance"].str.split(" ",expand=True)

dfdf.columns=["OTU1","OTU2","Distance",]


dfdf["Distance"]=dfdf["Distance"].astype(float)


#plot and save the final distribution of phylogebnetic distances we chose

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.hist(dfdf["Distance"], bins=20)
fig1.savefig(savepath+taxname+"/plots/distplot_"+taxname+"_uniform.png")

#save the pairs and tree branch lengths selected

df1str=dfdf.to_csv(header=True,sep="\t")
f= open(savepath+taxname+"/" + taxname+"_10k_99_dists.csv","w+")
f.write(df1str)
f.close()


#create the shortnames for the OTU-pairs

a=[]
b=[]
#if we have archaea:


if "Archaea" in taxname:

    for i in range(len(f2)):
        a.append(re.sub(r'^.*?_99_', 'A99_', f2["OTU1"].values[i]) + " " + re.sub(r'^.*?_99_', 'A99_', f2["OTU2"].values[i]))
       

    dataframe_pairs1=pd.DataFrame(columns=["Pairs"])

    dataframe_pairs1["Pairs"]=a

 


    dataframe_pairs1=dataframe_pairs1["Pairs"].str.split(" ", expand=True)

    dataframe_pairs1.columns=["OTU1","OTU2"]


    p=[]
    for i in range(len(f2)):
        p.append(re.sub(r'^.*?_99_', 'A99_', dataframe_pairs1["OTU1"].values[i]) + " " + re.sub(r'^.*?_99_', 'A99_', dataframe_pairs1["OTU2"].values[i]))
else:
        #For Bacteria
    for i in range(len(f2)):
        a.append(re.sub(r'^.*?_99_', 'B99_', f2["OTU1"].values[i]) + " " + re.sub(r'^.*?_99_', 'B99_', f2["OTU2"].values[i]))
       

    dataframe_pairs1=pd.DataFrame(columns=["Pairs"])

    dataframe_pairs1["Pairs"]=a




    dataframe_pairs1=dataframe_pairs1["Pairs"].str.split(" ", expand=True)


    dataframe_pairs1.columns=["OTU1","OTU2"]

    dataframe_pairs1

    p=[]
    for i in range(len(f2)):
        p.append(re.sub(r'^.*?_99_', 'B99_', dataframe_pairs1["OTU1"].values[i]) + " " + re.sub(r'^.*?_99_', 'B99_', dataframe_pairs1["OTU2"].values[i]))

dataframe_pairs=pd.DataFrame(columns=["Pairs"])

dataframe_pairs["Pairs"]=p




dataframe_pairs=dataframe_pairs["Pairs"].str.split(" ", expand=True)


dataframe_pairs.columns=["OTU1","OTU2"]

#save the OTU-pairs with short identifier

df1str=dataframe_pairs.to_csv(header=False, index=False, sep="\t")
f= open(savepath+taxname+"/" + taxname+"_pairs.tbl","w+")
f.write(df1str)
f.close()


#Get a file with the taxonomy of our OTUs
